Copyright (c) Microsoft Corporation. All rights reserved.

# Tutorial: Train your first model

## Connect to Workspace

In [20]:
from azureml.core import Workspace
#workspace = Workspace.from_config()
workspace = Workspace.get(name='zhenzhuUKSouth',
                subscription_id='e9b2ec51-5c94-4fa8-809a-dc1e695e4896',
                resource_group='zhenzhuUKSouth'
                )

## Create an experiment

In [21]:
from azureml.core import Experiment
experiment = Experiment(workspace, "MyExperiment")

#experiment = Experiment(workspace, "diabetes-expr-2")

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcb3c317710>: Failed to establish a new connection: [Errno 99] Cannot assign requested address',)': /history/v1.0/subscriptions/e9b2ec51-5c94-4fa8-809a-dc1e695e4896/resourceGroups/zhenzhuUKSouth/providers/Microsoft.MachineLearningServices/workspaces/zhenzhuUKSouth/experiments/MyExperiment
WARNING - Retrying (Retry(total=1, connect=1, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcb3c3172b0>: Failed to establish a new connection: [Errno 99] Cannot assign requested address',)': /history/v1.0/subscriptions/e9b2ec51-5c94-4fa8-809a-dc1e695e4896/resourceGroups/zhenzhuUKSouth/providers/Microsoft.MachineLearningServices/workspaces/zhenzhuUKSouth/experiments/MyExperiment
WARNING - Retrying (Retry(total=0, connect=0, read=3, redire

ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='uksouth.experiments.azureml.net', port=443): Max retries exceeded with url: /history/v1.0/subscriptions/e9b2ec51-5c94-4fa8-809a-dc1e695e4896/resourceGroups/zhenzhuUKSouth/providers/Microsoft.MachineLearningServices/workspaces/zhenzhuUKSouth/experiments/MyExperiment (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcb3c3176a0>: Failed to establish a new connection: [Errno 99] Cannot assign requested address',))

## Load data and prepare for training

In [ ]:
from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

x_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna()
y_df = x_df.pop("Y")

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=66)

UserErrorException: UserErrorException:
	Message: Execution failed with error message: ScriptExecutionException was caused by StreamAccessException.
  Unable to authenticate data access to '[REDACTED]'.
    AuthenticationException was caused by AzureIdentityAccessTokenResolutionException.
      Failed to get access token based on azure identity.
        Failed to get access token based on azure identity.
          Failed due to inner exception of type: SocketException
| session_id=d86c1acc-dd84-41b8-b533-130051197897 ErrorCode: ScriptExecution.StreamAccess.Authentication.AzureIdentityAccessTokenResolution.Unexpected
	InnerException 
Error Code: ScriptExecution.StreamAccess.Authentication.AzureIdentityAccessTokenResolution.Unexpected
Failed Step: 4c348943-a7e9-4092-90ec-7779e2003570
Error Message: ScriptExecutionException was caused by StreamAccessException.
  Unable to authenticate data access to 'https://azureopendatastorage.blob.core.windows.net/mlsamples/diabetes/*.parquet'.
    Aut

## Explore Data

In [4]:
x_df.head()

NameError: name 'x_df' is not defined

In [5]:
print(x_df.dtypes)
x_df.describe()

AGE      int64
SEX      int64
BMI    float64
BP     float64
S1       int64
S2     float64
S3     float64
S4     float64
S5     float64
S6       int64
dtype: object


,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000


## Train a model on Notebooks

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import math

alphas = [0.1, 2, 0.3, 4, 0.5, 6, 0.7, 8, 0.9, 1.0]

for alpha in alphas:
    run = experiment.start_logging()
    run.log("alpha_value", alpha)
    print("alpha_value", alpha )
    run = experiment.start_logging()
    model = Ridge(alpha=alpha)
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X=X_test)
    rmse = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))
    # run.log("rmse", rmse)
    print("rmse", rmse)
    
    model_name = "model_alpha_" + str(alpha) + ".pkl"
    filename = "outputs/" + model_name
    
    joblib.dump(value=model, filename=filename)
    
    # run.upload_file(name=model_name, path_or_stream=filename)
    # run.complete()


alpha_value 0.1


NameError: name 'X_train' is not defined

## Go to Experiments Tab in the Azure ML Workspace Menu
